# Installing Google Cloud dependencies

In [87]:
try:
    import google.cloud.service_usage_v1
except ImportError:
    print('You need to pip install google-cloud-service-usage')
    !pip install google-cloud-service-usage -q

In [88]:
try:
    import google.cloud.artifactregistry_v1
except ImportError:
    print('You need to pip install google-cloud-artifact-registry')
    !pip install google-cloud-artifact-registry -q

In [89]:
try:
    import google.cloud.devtools.cloudbuild
except ImportError:
    print('You need to pip install google-cloud-build')
    !pip install google-cloud-build

In [90]:
from google.cloud import aiplatform
from datetime import datetime
import pkg_resources
from IPython.display import Markdown as md
from google.cloud import service_usage_v1
from google.cloud.devtools import cloudbuild_v1
from google.cloud import artifactregistry_v1
from google.cloud import storage
from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np
import pandas as pd

# Setup

In [91]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'tymestack-439300'

In [92]:
REGION = 'us-central1'
EXPERIMENT = '01'
SERIES = '01'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'housing'
BQ_TABLE = 'housing'

BASE_IMAGE = "gcr.io/deeplearning-platform-release/xgboost-cpu"
SCRIPT_PATH = 'trainer/train.py'
DOCKERFILE = 'Dockerfile'

HEAD_COMPUTE = 'n1-standard-16'
WORKER_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-8'

In [93]:
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [94]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client(project=PROJECT_ID)
gcs = storage.Client(project=PROJECT_ID)
su_client = service_usage_v1.ServiceUsageClient()
ar_client = artifactregistry_v1.ArtifactRegistryClient()
cb_client = cloudbuild_v1.CloudBuildClient()

In [95]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{SERIES}/{EXPERIMENT}"
DIR = f"temp/{EXPERIMENT}"

In [96]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'652637980750-compute@developer.gserviceaccount.com'

# Traning with RAY (autoscaling)

Installing dependencies for RAY-based training

In [11]:
!pip install --upgrade xgboost_ray ray tensorboardX google.cloud.aiplatform

In [12]:
from google.cloud import aiplatform
import vertex_ray
from vertex_ray import AutoscalingSpec, Resources
import xgboost_ray as xgb_ray
from ray import tune
import pickle
import ray
import sys
from google.cloud import bigquery
from ray.air.config import RunConfig, ScalingConfig
from ray.air import CheckpointConfig
from ray.train.xgboost import XGBoostTrainer
import pandas as pd
from ray.tune.tuner import Tuner, TuneConfig
from ray.tune.search.bayesopt import BayesOptSearch

Initializing RAY runtime with 28 CPUs using for training

In [13]:
ray.shutdown()
ray.init(num_cpus=28)

2024-10-23 06:05:01,992	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.15
Ray version:,2.37.0
Dashboard:,http://127.0.0.1:8265


(train_xgboost_model pid=317638) /opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
(train_xgboost_model pid=317638)   warnings.warn(_RANGE_PYARROW_WARNING)
(train_xgboost_model pid=317638) /opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
(train_xgboost_model pid=317638)   warnings.warn(_RANGE_PYARROW_WARNING)
(train_xgboost_model pid=317638) /opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
(train_xgboost_model pid=3176

(train_xgboost_model pid=317637) 
(train_xgboost_model pid=317637) View detailed results here: tymestack-439300/01/01/models/20241023060440/XGBoostTrainer_2024-10-23_06-05-13
(train_xgboost_model pid=317637) To visualize your results with TensorBoard, run: `tensorboard --logdir /var/tmp/ray/session_2024-10-23_06-04-58_972716_317054/artifacts/2024-10-23_06-05-14/XGBoostTrainer_2024-10-23_06-05-13/driver_artifacts`
(train_xgboost_model pid=317638) 
(train_xgboost_model pid=317638) 
(train_xgboost_model pid=317638) Training started without custom configuration.
(train_xgboost_model pid=317637) 
(raylet) WARNING: 22 PYTHON worker processes have been started on node: 6788233764ed76c9515c74ccc9cdc6a23a07a67861ea56d815dd7b37 with address: 10.128.0.7. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
(train_xgboost_model pid=317638) View detailed results h

(XGBoostTrainer pid=317949) Started distributed worker processes: 
(XGBoostTrainer pid=317949) - (node_id=6788233764ed76c9515c74ccc9cdc6a23a07a67861ea56d815dd7b37, ip=10.128.0.7, pid=318047) world_rank=0, local_rank=0, node_rank=0
(XGBoostTrainer pid=317949) - (node_id=6788233764ed76c9515c74ccc9cdc6a23a07a67861ea56d815dd7b37, ip=10.128.0.7, pid=318054) world_rank=1, local_rank=1, node_rank=0
(train_xgboost_model pid=317637) /opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1. [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(train_xgboost_model pid=317637)   warnings.warn(_RANGE_PYARROW_WARNING) [repeated 3x ac

...

(train_xgboost_model pid=319158) 
(train_xgboost_model pid=319158) Training finished iteration 86 at 2024-10-23 06:06:04. Total running time: 9s
(train_xgboost_model pid=319158) ╭─────────────────────────────────────────╮
(train_xgboost_model pid=319158) │ Training result                         │
(train_xgboost_model pid=319158) ├─────────────────────────────────────────┤
(train_xgboost_model pid=319158) │ checkpoint_dir_name   checkpoint_000000 │
(train_xgboost_model pid=319158) │ time_this_iter_s                0.38331 │
(train_xgboost_model pid=319158) │ time_total_s                     5.8233 │
(train_xgboost_model pid=319158) │ training_iteration                   86 │
(train_xgboost_model pid=319158) │ train-rmse                      0.02511 │
(train_xgboost_model pid=319158) ╰─────────────────────────────────────────╯
(train_xgboost_model pid=319158) Training saved a checkpoint for iteration 86 at: (gcs)tymestack-439300/01/01/models/20241023060440/XGBoostTrainer_2024-10-23_06-0

(RayTrainWorker pid=319288) Checkpoint successfully created at: Checkpoint(filesystem=gcs, path=tymestack-439300/01/01/models/20241023060440/XGBoostTrainer_2024-10-23_06-05-53/XGBoostTrainer_dd558_00000_0_2024-10-23_06-05-55/checkpoint_000000)


(train_xgboost_model pid=319158) 
(train_xgboost_model pid=319158) Training completed after 86 iterations at 2024-10-23 06:06:05. Total running time: 10s


(train_xgboost_model pid=319158) Wrote the latest version of all result files and experiment state to 'tymestack-439300/01/01/models/20241023060440/XGBoostTrainer_2024-10-23_06-05-53' in 0.4389s.


(train_xgboost_model pid=319158) 
(train_xgboost_model pid=319158) Available metrics: OrderedDict([('train-rmse', 0.02510849421864081), ('timestamp', 1729663563), ('checkpoint_dir_name', 'checkpoint_000000'), ('should_checkpoint', True), ('done', True), ('training_iteration', 86), ('trial_id', 'dd558_00000'), ('date', '2024-10-23_06-06-04'), ('time_this_iter_s', 0.38330507278442383), ('time_total_s', 5.823301076889038), ('pid', 319238), ('hostname', 'instance-20241022-184641'), ('node_ip', '10.128.0.7'), ('config', {}), ('time_since_restore', 5.823301076889038), ('iterations_since_restore', 86), ('experiment_tag', '0')])


Creating RAY cluster on GCP UI

![RAY name](./images/ray_name.png)

Specifying head node

![RAY head](./images/ray_head.png)

Specifying worker node

![RAY head](./images/ray_worker.png)

After the cluster is created, access it using `vertex_ray.get_ray_cluster()`

In [14]:
vertex_ray.get_ray_cluster("projects/652637980750/locations/us-central1/persistentResources/cluster-20241023-123602")

Cluster(cluster_resource_name='projects/652637980750/locations/us-central1/persistentResources/cluster-20241023-123602', network='', reserved_ip_ranges=[], service_account=None, state='RUNNING', python_version='3.10', ray_version='2.33', head_node_type=Resources(machine_type='n1-standard-16', node_count=1, accelerator_type=None, accelerator_count=0, boot_disk_type='pd-standard', boot_disk_size_gb=100, custom_image=None, autoscaling_spec=None), worker_node_types=[Resources(machine_type='n1-standard-4', node_count=1, accelerator_type=None, accelerator_count=0, boot_disk_type='pd-standard', boot_disk_size_gb=100, custom_image=None, autoscaling_spec=AutoscalingSpec(min_replica_count=1, max_replica_count=3))], dashboard_address='1a538065013652cd-dot-us-central1.aiplatform-training.googleusercontent.com', ray_metric_enabled=True, ray_logs_enabled=True, psc_interface_config=None, labels={})

In [15]:
print("Cluster Resources:", ray.cluster_resources())

Cluster Resources: {'memory': 9352657307.0, 'object_store_memory': 4676328652.0, 'CPU': 28.0, 'node:10.128.0.7': 1.0, 'node:__internal_head__': 1.0}


## Data Loading Function for BigQuery

```python
def load_data(project, dataset, table):
   """Load data from BigQuery and restructure into features and target"""
```

This function handles data loading and preprocessing:
- Fetches data from specified BigQuery table
- Restructures data by pairing even/odd rows
- Creates features by combining:
 - All columns from even rows
 - First 2 columns from odd rows
- Uses last column of odd rows as target
- Converts to Ray Dataset format for distributed training

Parameters:
- `project`: GCP project ID
- `dataset`: BigQuery dataset name
- `table`: BigQuery table name

Returns a Ray Dataset with features and target column ready for training.

In [17]:
def load_data(project, dataset, table):
    """
    Load data from BigQuery and restructure into features and target
    with handling for mismatched row counts
    """
    client = bigquery.Client(project=project)
    query = f"SELECT * FROM `{project}.{dataset}.{table}`"
    raw_df = client.query(query).to_dataframe()
    
    # Convert to numpy for easier array manipulation
    raw_values = raw_df.values
    
    # Get even and odd indices, ensuring we have matching pairs
    n_pairs = min(len(raw_values[::2]), len(raw_values[1::2]))
    even_rows = raw_values[::2][:n_pairs]
    odd_rows = raw_values[1::2][:n_pairs]
    
    # Now combine features with guaranteed matching dimensions
    features = np.hstack([
        even_rows,  # all columns from even rows
        odd_rows[:, :2]  # first 2 columns from odd rows
    ])
    
    # Get target from last column of odd rows
    target = odd_rows[:, 2]
    
    # Convert back to DataFrame for proper column handling
    feature_cols = [f'feat_{i}' for i in range(features.shape[1])]
    X = pd.DataFrame(features, columns=feature_cols)
    y = pd.Series(target, name='target')
    
    # Ensure data types are correct
    X = X.select_dtypes(include=['number']).fillna(0)
    y = y.astype(float)
    
    # Convert to Ray Dataset
    dataset = ray.data.from_pandas(pd.concat([X, y], axis=1))
    return dataset

## XGBoost Model Training Function

This function implements the training loop for XGBoost using Ray's distributed training capabilities:

```python
def train_xgboost_model(config):
    """Training function using Ray's XGBoostTrainer with distributed processing"""
```

Key components:
- Loads data from BigQuery
- Configures XGBoost with hyperparameters from tuning
- Uses 2 workers for distributed training
- Saves checkpoints for model versioning
- Returns MSE, RMSE metrics and model checkpoint

The function takes a `config` dictionary containing hyperparameters:
- n_estimators: Number of boosting rounds
- max_depth: Maximum tree depth
- learning_rate: Learning rate for gradient descent
- subsample: Subsample ratio for training data

In [18]:
def train_xgboost_model(config):
    """Training function using Ray's XGBoostTrainer with restructured data"""
    # Load and preprocess data
    train_dataset = load_data(BQ_PROJECT, BQ_DATASET, BQ_TABLE)
    
    trainer = XGBoostTrainer(
        scaling_config=ScalingConfig(
            num_workers=2,
            use_gpu=False,
        ),
        run_config=RunConfig(
            checkpoint_config=CheckpointConfig(
                num_to_keep=None  # Keep all checkpoints
            ),
            storage_path=f"{URI}/models/{TIMESTAMP}",
        ),
        label_column="target",
        num_boost_round=config["n_estimators"],
        params={
            "objective": "reg:squarederror",
            "max_depth": config["max_depth"],
            "learning_rate": config["learning_rate"],
            "subsample": config["subsample"],
            "tree_method": "hist",
            "eval_metric": ["rmse"],
        },
        datasets={"train": train_dataset},
    )
    
    result = trainer.fit()
    
    # Extract metrics - updated to handle new metrics structure
    try:
        # First try the new metrics structure
        final_rmse = result.metrics["eval_metrics"]["rmse"][-1]
    except KeyError:
        try:
            # Try alternative metrics structure
            final_rmse = result.metrics["rmse"][-1]
        except KeyError:
            # If both fail, print available metrics and use a default
            print("Available metrics:", result.metrics)
            final_rmse = result.metrics.get("training_iteration", 0)
    
    final_mse = final_rmse ** 2 if final_rmse is not None else float('inf')
    
    return {
        "mse": final_mse,
        "rmse": final_rmse,
        "model": result.checkpoint
    }

## Hyperparameter Tuning with Bayesian Optimization

```python
tuner = Tuner(
   train_xgboost_model,
   param_space=tune_config,
   tune_config=TuneConfig(...)
)
```

Configures and runs hyperparameter optimization using Ray Tune:
- Uses Bayesian Optimization (`BayesOptSearch`) for intelligent parameter search
- Performs 5 trials to find optimal parameters
- Runs 2 trials concurrently for faster tuning
- Optimizes for minimum RMSE

The tuner:
1. Explores the parameter space defined in `tune_config`
2. Tracks performance across trials
3. Returns best performing model configuration
4. Prints optimal hyperparameters and RMSE score

This optimization helps find the best combination of:
- Number of estimators
- Maximum tree depth
- Learning rate
- Subsample ratio

In [19]:
# Hyperparameter space
tune_config = {
    "n_estimators": tune.choice([80, 85]),
    "max_depth": tune.choice([5, 6]),
    "learning_rate": tune.loguniform(0.05, 0.1),
    "subsample": tune.uniform(0.8, 0.9)
}

In [20]:
# Create the tuner
tuner = Tuner(
    train_xgboost_model,
    param_space=tune_config,
    tune_config=TuneConfig(
        num_samples=5,           # Number of trials to run
        metric="rmse",          # Metric to optimize
        mode="min",             # Minimize the metric
        max_concurrent_trials=2,  # 2-way concurrent tuning
        search_alg=BayesOptSearch()
    ),
)

# Run the hyperparameter search
result_grid = tuner.fit()

# Get the best result
best_result = result_grid.get_best_result()

print("Best hyperparameters found:", best_result.config)
print("Best RMSE achieved:", best_result.metrics["rmse"])

(pid=318204) Running 0: 0.00 row [00:00, ? row/s]

(pid=318208) Running 0: 0.00 row [00:00, ? row/s]

(pid=318971) Running 0: 0.00 row [00:00, ? row/s]

(pid=318974) Running 0: 0.00 row [00:00, ? row/s]

(pid=319365) Running 0: 0.00 row [00:00, ? row/s]

2024-10-23 06:06:06,006	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/jupyter/ray_results/train_xgboost_model_2024-10-23_06-05-04' in 0.0110s.
2024-10-23 06:06:06,024	INFO tune.py:1041 -- Total run time: 61.89 seconds (61.81 seconds for the tuning loop).


Best hyperparameters found: {'n_estimators': 85, 'max_depth': 5, 'learning_rate': 0.06838712273668071, 'subsample': 0.8455691366345284}
Best RMSE achieved: 86


# Upload Container Image

In [180]:
import subprocess

# Get a list of all installed packages and their versions
installed_packages = subprocess.run(['pip', 'freeze'], capture_output=True, text=True).stdout.splitlines()

# List to store only valid packages
valid_packages = []

# Iterate through each package and attempt to install it to check for issues
for package in installed_packages:
    package_name = package.split('==')[0]  # Get only the package name (exclude version)
    
    try:
        print(f"Checking package: {package_name}")
        # Try installing the package to check if it's valid
        subprocess.run(['pip', 'install', package_name], check=True, capture_output=True, text=True)
        valid_packages.append(package)  # If installation succeeds, add to valid list
    except subprocess.CalledProcessError as e:
        print(f"Skipping package {package_name} due to error: {e}")
    except OSError as os_error:
        print(f"Skipping package {package_name} due to missing files or directories: {os_error}")

# Write the valid packages to a requirements.txt file
with open('requirements.txt', 'w') as f:
    f.write("\n".join(valid_packages))

print("requirements.txt file has been created with valid installed libraries and versions.")

requirements.txt file has been created with all installed libraries and versions.


In [194]:
bucket = gcs.lookup_bucket(PROJECT_ID)
SOURCEPATH = f'{SERIES}/{EXPERIMENT}/training'

In [195]:
blob = bucket.blob(f'{SOURCEPATH}/{EXPERIMENT}_trainer/train.py')
blob.upload_from_filename(SCRIPT_PATH)

In [196]:
dockerfile = f"""
FROM gcr.io/deeplearning-platform-release/xgboost-cpu
WORKDIR /training
# copy requirements and install them
COPY requirements.txt ./
RUN pip install --no-cache-dir --upgrade pip \
  && pip install --no-cache-dir --ignore-installed --no-deps -r requirements.txt || true
"""

In [198]:
blob = bucket.blob(f'{SOURCEPATH}/requirements.txt')
blob.upload_from_filename('requirements.txt')

In [199]:
blob = bucket.blob(f'{SOURCEPATH}/Dockerfile')
blob.upload_from_string(dockerfile)

In [200]:
for repo in ar_client.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    print(repo.name)

projects/tymestack-439300/locations/us-central1/repositories/tymestack-439300


In [201]:
docker_repo = None
for repo in ar_client.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    if f'{PROJECT_ID}' == repo.name.split('/')[-1]:
        docker_repo = repo
        print(f'Retrieved existing repo: {docker_repo.name}')

if not docker_repo:
    operation = ar_client.create_repository(
        request = artifactregistry_v1.CreateRepositoryRequest(
            parent = f'projects/{PROJECT_ID}/locations/{REGION}',
            repository_id = f'{PROJECT_ID}',
            repository = artifactregistry_v1.Repository(
                description = f'A repository for the {EXPERIMENT} experiment that holds docker images.',
                name = f'{PROJECT_ID}',
                format_ = artifactregistry_v1.Repository.Format.DOCKER,
                labels = {'series': SERIES, 'experiment': EXPERIMENT}
            )
        )
    )
    print('Creating Repository ...')
    docker_repo = operation.result()
    print(f'Completed creating repo: {docker_repo.name}')

Retrieved existing repo: projects/tymestack-439300/locations/us-central1/repositories/tymestack-439300


In [202]:
docker_repo.name, docker_repo.format_.name

('projects/tymestack-439300/locations/us-central1/repositories/tymestack-439300',
 'DOCKER')

In [203]:
REPOSITORY = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{docker_repo.name.split('/')[-1]}"

In [204]:
# setup the build config with empty list of steps - these will be added sequentially
build = cloudbuild_v1.Build(
    steps = []
)
# retrieve the source
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/gsutil',
        'args': ['cp', '-r', f'gs://{PROJECT_ID}/{SOURCEPATH}/*', '/workspace']
    }
)
# docker build
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/docker',
        'args': ['build', '-t', f'{REPOSITORY}/{EXPERIMENT}_trainer', '/workspace']
    }    
)
# docker push
build.images = [f"{REPOSITORY}/{EXPERIMENT}_trainer"]

In [205]:
build

steps {
  name: "gcr.io/cloud-builders/gsutil"
  args: "cp"
  args: "-r"
  args: "gs://tymestack-439300/01/01/training/*"
  args: "/workspace"
}
steps {
  name: "gcr.io/cloud-builders/docker"
  args: "build"
  args: "-t"
  args: "us-central1-docker.pkg.dev/tymestack-439300/tymestack-439300/01_trainer"
  args: "/workspace"
}
images: "us-central1-docker.pkg.dev/tymestack-439300/tymestack-439300/01_trainer"

In [206]:
operation = cb_client.create_build(
    project_id = PROJECT_ID,
    build = build
)

In [207]:
response = operation.result()
response.status, response.artifacts

(<Status.SUCCESS: 3>,
 images: "us-central1-docker.pkg.dev/tymestack-439300/tymestack-439300/01_trainer")

# Deploy Best Model's Endpoint

In [35]:
!pip install py-ubjson

  Using cached py-ubjson-0.16.1.tar.gz (50 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for py-ubjson: filename=py_ubjson-0.16.1-cp310-cp310-linux_x86_64.whl size=45683 sha256=518ace72394477a125c517c43d2bf374a9fbe970833f162bd51282c4808020d0
  Stored in directory: /home/jupyter/.cache/pip/wheels/ca/7d/41/4ca3da5703c6c538acebd3eba359d8226dc7e1c4494480a2d2
Successfully built py-ubjson


In [36]:
path = f"gcs://{best_result.metrics['model'].path}"
path

'gcs://tymestack-439300/01/01/models/20241023060440/XGBoostTrainer_2024-10-23_06-05-13/XGBoostTrainer_c5561_00000_0_2024-10-23_06-05-14/checkpoint_000000'

In [ ]:
import ubjson
import pickle
from google.cloud import storage

# Initialize GCS client
client = storage.Client()

# Original path
# path = 'gcs://tymestack-439300/01/01/models/20241023060440/XGBoostTrainer_2024-10-23_06-05-13/XGBoostTrainer_c5561_00000_0_2024-10-23_06-05-14/checkpoint_000000'
# Find the index where "01" starts and slice from there
source_blob_name = path[path.index("01"):]
destination_file_name = "/tmp/model.ubj"  # Temporary local file path

# Download file from GCS
bucket = client.bucket(BUCKET)
blob = bucket.blob(source_blob_name)
blob.download_to_filename(destination_file_name)

print(f"Downloaded {source_blob_name} to {destination_file_name}.")

# Load the UBJ model from the local temporary path
with open(destination_file_name, "rb") as ubj_file:
    model = ubjson.load(ubj_file)

# Save the model as a pickle (.pkl) file
pkl_file_path = "/tmp/model.pkl"
with open(pkl_file_path, "wb") as pkl_file:
    pickle.dump(model, pkl_file)

print(f"Model has been successfully converted and saved as {pkl_file_path}.")

In [40]:
blob = bucket.blob(f"{source_blob_name}/model.pkl")
blob.upload_from_filename(pkl_file_path)

In [100]:
from google.cloud import storage

# Initialize GCS client
storage_client = storage.Client()

# Parse the GCS path
gcs_path = path.replace('gcs://', '')
bucket_name = gcs_path.split('/')[0]
blob_prefix = '/'.join(gcs_path.split('/')[1:])

# Get bucket
bucket = storage_client.bucket(bucket_name)

# List contents
print(f"Listing contents of checkpoint directory: {path}")
blobs = list(bucket.list_blobs(prefix=blob_prefix))
for blob in blobs:
    print(f"- {blob.name}")

Listing contents of checkpoint directory: gcs://tymestack-439300/01/01/models/20241023060440/XGBoostTrainer_2024-10-23_06-05-13/XGBoostTrainer_c5561_00000_0_2024-10-23_06-05-14/checkpoint_000000
- 01/01/models/20241023060440/XGBoostTrainer_2024-10-23_06-05-13/XGBoostTrainer_c5561_00000_0_2024-10-23_06-05-14/checkpoint_000000/
- 01/01/models/20241023060440/XGBoostTrainer_2024-10-23_06-05-13/XGBoostTrainer_c5561_00000_0_2024-10-23_06-05-14/checkpoint_000000/model.pkl
- 01/01/models/20241023060440/XGBoostTrainer_2024-10-23_06-05-13/XGBoostTrainer_c5561_00000_0_2024-10-23_06-05-14/checkpoint_000000/model.ubj


In [208]:
# Upload the model to Vertex AI
model = aiplatform.Model.upload(
    display_name="xgboost-housing",
    artifact_uri=f"gs://{BQ_PROJECT}/" + source_blob_name.rsplit("/", 1)[0],
    serving_container_image_uri=f"{REPOSITORY}/{EXPERIMENT}_trainer",
)

Creating Model


INFO:google.cloud.aiplatform.models:Creating Model


Create Model backing LRO: projects/652637980750/locations/us-central1/models/1723547148701663232/operations/3876225611185782784


INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/652637980750/locations/us-central1/models/1723547148701663232/operations/3876225611185782784


Model created. Resource name: projects/652637980750/locations/us-central1/models/1723547148701663232@1


INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/652637980750/locations/us-central1/models/1723547148701663232@1


To use this Model in another session:


INFO:google.cloud.aiplatform.models:To use this Model in another session:


model = aiplatform.Model('projects/652637980750/locations/us-central1/models/1723547148701663232@1')


INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/652637980750/locations/us-central1/models/1723547148701663232@1')


# Deploy model

In [209]:
DEPLOYED_NAME = f"ray-on-vertex-registered-xgboost-model-{aiplatform.utils.timestamped_unique_name()}"

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=False,
)

In [210]:
endpoint

<google.cloud.aiplatform.models.Endpoint object at 0x7f34ab733760> is waiting for upstream dependencies to complete.

In [211]:
from google.cloud import aiplatform_v1
# Initialize the Vertex AI client
client = aiplatform_v1.EndpointServiceClient(client_options={"api_endpoint": f"{REGION}-aiplatform.googleapis.com"})
# Define the parent resource (where the endpoints are located)
parent = f"projects/{BQ_PROJECT}/locations/{REGION}"
# List all available endpoints
endpoints = client.list_endpoints(parent=parent)
# List all available endpoints
request = aiplatform_v1.ListEndpointsRequest(parent=parent)
response = client.list_endpoints(request=request)

# Print the details of each endpoint
for endpoint in response:
    print(f"Endpoint name: {endpoint.name}")
    print(f"Display name: {endpoint.display_name}")
    print(f"Deployed models: {len(endpoint.deployed_models)}")
    print("--------")

Endpoint name: projects/652637980750/locations/us-central1/endpoints/7890958557548380160
Display name: xgboost-housing_endpoint
Deployed models: 0
--------
Endpoint name: projects/652637980750/locations/us-central1/endpoints/6044201235349766144
Display name: xgboost-housing_endpoint
Deployed models: 0
--------
Endpoint name: projects/652637980750/locations/us-central1/endpoints/2759106782159699968
Display name: xgboost-housing_endpoint
Deployed models: 1
--------
Endpoint name: projects/652637980750/locations/us-central1/endpoints/7815523263789924352
Display name: xgboost-housing_endpoint
Deployed models: 0
--------
Endpoint name: projects/652637980750/locations/us-central1/endpoints/8116982963847036928
Display name: xgboost-housing_endpoint
Deployed models: 0
--------


In [176]:
endpoint = aiplatform.Endpoint('projects/652637980750/locations/us-central1/endpoints/8116982963847036928')